In [1]:
!docker run -p 6333:6333 -p 6334:6334 -v "$(pwd)/qdrant_storage:/qdrant/storage:z" qdrant/qdrant

docker: Error response from daemon: create $(pwd)/qdrant_storage: "$(pwd)/qdrant_storage" includes invalid characters for a local volume name, only "[a-zA-Z0-9][a-zA-Z0-9_.-]" are allowed. If you intended to pass a host directory, use absolute path

Run 'docker run --help' for more information


I have created a volume named qdrant_deep_dive. i will mount this volume in a container running our Qdrant

In [2]:
!docker run -d --name qdrant_container -p 6333:6333 -p 6334:6334 -v qdrant_deep_dive:/qdrant/storage qdrant/qdrant:latest

8ed88de7302316d80b68187101935546e9e89b52ed655033419e0b5d2040a6a0


In [1]:
from qdrant_client import QdrantClient, models

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

client = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))


In [3]:
print(os.getenv("QDRANT_URL"))

https://928a1c3a-0459-4ee8-8a20-61272372b50d.us-east-1-1.aws.cloud.qdrant.io


In [ ]:
# client = QdrantClient(":memory:") # For local Qdrant instance in memory, with no data persistence

# Create a collection

A collection in Qdrant is analogous to a table in a relationial DB. It's a container for storing vectors. For creating a collection we need to specify:
- Name: A unique identifier for the collection
- Vector Configuration:
    - Size: The dimensionality of the vectors
    - Distance Metric: The method to measure similarity between vectors.

In [6]:
from qdrant_client.models import Distance, VectorParams

collection_name = "my_first_collection"

client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=4, # dimensionality of the vectors
        distance=Distance.COSINE # distance metric for similarity search to use
        ),
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `my_first_collection` already exists!"},"time":0.020477233}'

## Verify Collection Creation

In [7]:
# Retrieve and display the list of collections
collections = client.get_collections()
print("Existing collections:", collections)

Existing collections: collections=[CollectionDescription(name='my_first_collection')]


Points are the core data entities in Qdrant. Each point contains:

- ID: A unique identifier
- Vector Data: An array of numerical values representing the data point in vector space
- Payload (Optional): Additional metadata

In [8]:
# Define the vectors to be inserted
points = [
    models.PointStruct(
        id=1,
        vector=[0.1, 0.2, 0.3, 0.4],  # 4D vector
        payload={"category": "example"}  # Metadata (optional)
    ),
    models.PointStruct(
        id=2,
        vector=[0.2, 0.3, 0.4, 0.5],
        payload={"category": "demo"}
    )
]

# Insert vectors into the collection
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

##  Retrieve Collection Details

In [9]:
collection_info = client.get_collection(collection_name)
print("Collection info:", collection_info)

Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> warnings=None indexed_vectors_count=0 points_count=2 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=4, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead

## Run Your First Similarity Search

Find the most similar vector to a given query using Qdrant’s search capabilities:

How Similarity Search Works:

Qdrant searches the collection to find the vectors that are closest to your query vector.
The results are ranked by their similarity score, with the best matches appearing first.

In [10]:
query_vector = [0.08, 0.14, 0.33, 0.28]

search_results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=1  # Return the top 1 most similar vector
)

print("Search results:", search_results)

Search results: points=[ScoredPoint(id=1, version=4, score=0.97642946, payload={'category': 'example'}, vector=None, shard_key=None, order_value=None)]


### Trying out filtered search

In this case, our result must contain the payload where the field category is 'example'. This means QDRANT will only return the point if it satisfies the filter condition.

Qdrant Cloud requires explicit payload indexes for efficient filtering on metadata fields. The index type KEYWORD is used for exact string matching

In [11]:
from qdrant_client.models import PayloadSchemaType

# Create a payload index for the "category" field
client.create_payload_index(
    collection_name=collection_name,
    field_name="category",
    field_schema=PayloadSchemaType.KEYWORD
)

print("Payload index created for 'category' field")

Payload index created for 'category' field


In [12]:
search_filter = models.Filter(
    must=[
        models.FieldCondition(
            key="category",
            match=models.MatchValue(value="example") # Only return vectors where category == "example"
        )
    ]
)

filtered_search_results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    query_filter=search_filter,
    limit=1
)

print("Filtered search results:", filtered_search_results)

Filtered search results: points=[ScoredPoint(id=1, version=4, score=0.97642946, payload={'category': 'example'}, vector=None, shard_key=None, order_value=None)]
